In [1]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd

In [2]:
from datasets import load_dataset

In [3]:
import re

In [4]:
model = SentenceTransformer('all-mpnet-base-v2')

# Squad dataset

In [5]:
sq = load_dataset('squad')

Reusing dataset squad (C:\Users\Intag\.cache\huggingface\datasets\squad\plain_text\1.0.0\d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
sq['train'][4]['question']

'What sits on top of the Main Building at Notre Dame?'

In [7]:
sq['train'][4]['answers']['text'][0]

'a golden statue of the Virgin Mary'

In [8]:
df = pd.read_csv('./eval_data.csv')

In [9]:
df.columns

Index(['multilabel_id', 'query', 'filters', 'gold_answers', 'answer',
       'context', 'exact_match', 'f1', 'rank', 'document_id',
       'gold_document_ids', 'offsets_in_document', 'gold_offsets_in_documents',
       'type', 'node', 'eval_mode', 'index'],
      dtype='object')

In [10]:
df = df.drop(columns = ['f1', 'rank', 'document_id',
       'gold_document_ids', 'offsets_in_document', 'gold_offsets_in_documents',
       'type', 'node', 'eval_mode', 'index'])

# Preprocessing

In [11]:
import ast

In [12]:
df = df.fillna('not_given')

In [13]:
correct = []
for i in range(len(df['gold_answers'])):
    try:
        temp = ast.literal_eval(df['gold_answers'][i])
    except:
        pass
    df['answer'][i] = re.sub(r'[^\w\s]', '', df['answer'][i])
    df['answer'][i] = df['answer'][i].lower()
    for j in range(len(temp)):
        temp[j] = re.sub(r'[^\w\s]', '', temp[j])
        temp[j] = temp[j].lower()
    correct.append(temp)

<ipython-input-13-e8d162cfe9c9>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['answer'][i] = re.sub(r'[^\w\s]', '', df['answer'][i])
<ipython-input-13-e8d162cfe9c9>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['answer'][i] = df['answer'][i].lower()


In [14]:
df['gold_answers'] = correct

In [15]:
df

,multilabel_id,query,filters,gold_answers,answer,context,exact_match
0,5616814845033682107,who is written in the book of life,b'null',"[all people considered righteous before god, e...",not_given,not_given,0.0
1,5616814845033682107,who is written in the book of life,b'null',"[all people considered righteous before god, e...",those whose names are written in the book of l...,"ohn of Patmos. As described, only those whose ...",0.0
2,5616814845033682107,who is written in the book of life,b'null',"[all people considered righteous before god, e...",only the names of the righteous,. The Psalmist likewise speaks of the Book of ...,0.0
3,5616814845033682107,who is written in the book of life,b'null',"[all people considered righteous before god, e...",those who are found written in the book and wh...,those who are found written in the book and wh...,0.0
4,-4885963241536934817,who was the girl in the video brenda got a baby,b'null',[ethel edy proctor],her cousin,ng a story in the newspaper of a 12-year-old g...,0.0
...,...,...,...,...,...,...,...
95,-2582649975336248842,the fertile crescent is located between what t...,b'null',[tigris and euphrates rivers],not_given,not_given,0.0
96,2968376238709049184,how did the fertile crescent get its nickname,b'null',[popularized by university of chicago archaeol...,not_given,not_given,0.0
97,2968376238709049184,how did the fertile crescent get its nickname,b'null',[popularized by university of chicago archaeol...,the cradle of civilization,atelevel societies. This has earned the region...,0.0
98,2968376238709049184,how did the fertile crescent get its nickname,b'null',[popularized by university of chicago archaeol...,the cradle of civilization,e Fertile Crescent in red The Fertile Crescent...,0.0


In [16]:
def preprocessor(answer):
    try:
        answer = ast.literal_eval(answer)
    except:
        pass
    if (isinstance(answer, list) == True):
        for i in range(len(answer)):
            answer[i] = re.sub(r'[^\w\s]', '',answer[i])
            answer[i] = answer[i].lower()
        return answer
    else:
        answer = re.sub(r'[^\w\s]', '', answer)
        answer = answer.lower()
        return answer

# Evaluation

In [17]:
def evaluation(true_ans, user_ans):
    if (isinstance(true_ans, list) == True):
        marks = []
        for i in range(len(true_ans)):
            true_embed = model.encode(true_ans[i], convert_to_tensor=True)
            user_embed = model.encode(user_ans, convert_to_tensor=True) 
            cosine_scores = util.pytorch_cos_sim(true_embed, user_embed)
            score = round(cosine_scores.item()*10)
            marks.append(score)
        if (max(marks) > 9):
            return 10
        else:
            return max(marks)
    else:
        true_embed = model.encode(true_ans, convert_to_tensor=True)
        user_embed = model.encode(user_ans, convert_to_tensor=True)
        cosine_scores = util.pytorch_cos_sim(true_embed, user_embed)
        return round(cosine_scores.item()*10)

In [18]:
for i in range(len(df)):
    marks = evaluation(df['gold_answers'][i], df['answer'][i])
    print('Correct answer:',df['gold_answers'][i])
    print('User answer:',df['answer'][i])
    print('Marks:',marks)      

Correct answer: ['all people considered righteous before god', 'every person who is destined for heaven or the world to come']
User answer: not_given
Marks: 1
Correct answer: ['all people considered righteous before god', 'every person who is destined for heaven or the world to come']
User answer: those whose names are written in the book of life from the foundation of the world
Marks: 6
Correct answer: ['all people considered righteous before god', 'every person who is destined for heaven or the world to come']
User answer: only the names of the righteous
Marks: 5
Correct answer: ['all people considered righteous before god', 'every person who is destined for heaven or the world to come']
User answer: those who are found written in the book and who shall escape the troubles preparatory to the coming of the messianic kingdom are they who together with the risen martyrs
Marks: 5
Correct answer: ['ethel edy proctor']
User answer: her cousin
Marks: 3
Correct answer: ['ethel edy proctor']


Correct answer: ['the 90s', 'around 2011']
User answer: not_given
Marks: 1
Correct answer: ['the 90s', 'around 2011']
User answer: 2011
Marks: 7
Correct answer: ['the 90s', 'around 2011']
User answer: 20150119 3j5210 152 finn puts a spell on the compound trapping wolves and vampires klaus takes cami to the safe house then confronts kol about rebekahs whereabouts at the end of the episode it is revealed that the ghost who talked to rebekah is freya 33 11 brotherhood of the damned sylvain white kyle arrington  diane ademujohn january 26 2015
Marks: 1
Correct answer: ['the 90s', 'around 2011']
User answer: 32 10 gonna set your flag on fire rob hardy ashley lyle  bart nickerson january 19 2015
Marks: 3
Correct answer: ['an apprentice named jeanpaul valley aka azrael', 'jeanpaul valley aka azrael']
User answer: not_given
Marks: 2
Correct answer: ['an apprentice named jeanpaul valley aka azrael', 'jeanpaul valley aka azrael']
User answer: zoom
Marks: 2
Correct answer: ['an apprentice named j

# Using squad

In [29]:
print('Question:',preprocessor(sq['train'][4]['question']), '?')
print('Correct answer:',preprocessor(sq['train'][4]['answers']['text'][0]),'.')
print('Student answer:','statue of Virgin Mary')
print('Marks:',evaluation(sq['train'][4]['answers']['text'][0], 'statue of Virgin Mary'))

Question: what sits on top of the main building at notre dame ?
Correct answer: a golden statue of the virgin mary .
Student answer: statue of Virgin Mary
Marks: 9
